In [1]:
import sys
sys.path.append('../')

from models import utils, Simple_CNN, Simple_LSTM, Hierarchical

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

import matplotlib.pyplot as plt

import numpy as np
from importlib import reload

import yaml

In [2]:
with open("./yamls/Hierarchy.yaml", "r") as f:
  net_params = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
net_organization = net_params["net_organization"]

net_organization

{'phylum': {'class_Arthropoda': ['family_Arachnida', 'family_Insecta'],
  'class_Chordata': ['family_Amphibia',
   'family_Aves',
   'family_Mammalia',
   'family_Reptilia']}}

In [4]:
node_params = net_params["node_params"]
node_params

{'phylum': {'type': 'CNN',
  'epochs': 5,
  'lr': 0.001,
  'order': [],
  'log_file': 'test_train.log'},
 'class_Arthropoda': {'type': 'CNN',
  'epochs': 5,
  'lr': 0.001,
  'order': ['Arthropoda'],
  'log_file': 'test_train.log'},
 'family_Arachnida': {'type': 'CNN',
  'epochs': 5,
  'lr': 0.001,
  'order': ['Arthropoda', 'Arachnida'],
  'log_file': 'test_train.log'},
 'family_Insecta': {'type': 'CNN',
  'epochs': 5,
  'lr': 0.001,
  'order': ['Arthropoda', 'Insecta'],
  'log_file': 'test_train.log'},
 'class_Chordata': {'type': 'CNN',
  'epochs': 5,
  'lr': 0.001,
  'order': ['Chordata'],
  'log_file': 'test_train.log'},
 'family_Actinopterygii': {'type': 'CNN',
  'epochs': 5,
  'lr': 0.001,
  'order': ['Chordata', 'Actinopterygii'],
  'log_file': 'test_train.log'},
 'family_Amphibia': {'type': 'CNN',
  'epochs': 5,
  'lr': 0.001,
  'order': ['Chordata', 'Amphibia'],
  'log_file': 'test_train.log'},
 'family_Aves': {'type': 'CNN',
  'epochs': 5,
  'lr': 0.001,
  'order': ['Chordata',

In [5]:
net_architecture = net_params["net_architecture"]

net_architecture

{'phylum': {'input_shape': [3, 1024, 256],
  'conv_channels': [8, 16, 32, 16, 8, 4],
  'conv_kernel_size': [2, 2, 2, 2, 2, 2],
  'pooling_size': [[2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2]],
  'conv_dropout': [0.0, 0.0, 0.2, 0.2, 0.0, 0.0],
  'linear_features': [64, 2],
  'linear_dropout': [0.4],
  'last_layer_activation': 'Sigmoid()'},
 'class_Arthropoda': {'input_shape': [3, 1024, 256],
  'conv_channels': [8, 16, 32, 16, 8, 4],
  'conv_kernel_size': [2, 2, 2, 2, 2, 2],
  'pooling_size': [[2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2]],
  'conv_dropout': [0.0, 0.0, 0.2, 0.2, 0.0, 0.0],
  'linear_features': [64, 2],
  'linear_dropout': [0.4],
  'last_layer_activation': 'Sigmoid()'},
 'family_Arachnida': {'input_shape': [3, 1024, 256],
  'conv_channels': [8, 16, 32, 16, 8, 4],
  'conv_kernel_size': [2, 2, 2, 2, 2, 2],
  'pooling_size': [[2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2]],
  'conv_dropout': [0.0, 0.0, 0.2, 0.2, 0.0, 0.0],
  'linear_features': [64, 1],
  'linear_dropout':

In [6]:
# Parameters
random_seed = 42

net_name = "test"

In [91]:
reload(utils)

transform_img = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
    ]
)

transform_labels = transforms.Compose(
    [transforms.ToTensor()]
)

dataset = utils.HierarchicalDataset("../datasets/smallDatasetBroad.csv",
                                    "../spectrograms/small_dataset_broad",
                                    "../mfcc/",
                                    "gbifID",
                                    ["phylum", "class", "family"],
                                    transform=transform_img, preload=False)
dataset.csv

,gbifID,species,genus,family,class,phylum,file_name
0,1230246828,Pycnonotus nigricans,Pycnonotus,Pycnonotidae,Aves,Chordata,Chordata/Aves/Pycnonotidae/Pycnonotus/Pycnonot...
1,779853992,Emberiza hortulana,Emberiza,Emberizidae,Aves,Chordata,Chordata/Aves/Emberizidae/Emberiza/Emberiza_ho...
2,779849670,Cervus elaphus,Cervus,Cervidae,Mammalia,Chordata,Chordata/Mammalia/Cervidae/Cervus/Cervus_elaph...
3,1269848168,Numenius phaeopus,Numenius,Scolopacidae,Aves,Chordata,Chordata/Aves/Scolopacidae/Numenius/Numenius_p...
4,779860246,Neofelis nebulosa,Neofelis,Felidae,Mammalia,Chordata,Chordata/Mammalia/Felidae/Neofelis/Neofelis_ne...
...,...,...,...,...,...,...,...
1995,779852839,Dryobates minor,Dryobates,Picidae,Aves,Chordata,Chordata/Aves/Picidae/Dryobates/Dryobates_mino...
1996,991883728,Coccothraustes coccothraustes,Coccothraustes,Fringillidae,Aves,Chordata,Chordata/Aves/Fringillidae/Coccothraustes/Cocc...
1997,779865264,Rallus aquaticus,Rallus,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Rallus/Rallus_aquaticus...
1998,1229953095,Colinus virginianus,Colinus,Odontophoridae,Aves,Chordata,Chordata/Aves/Odontophoridae/Colinus/Colinus_v...


In [128]:
reload(Hierarchical)

torch.cuda.empty_cache()
reload(Hierarchical)
H = Hierarchical.HierarchicalClassification(net_architecture, node_params, net_organization, dataset)

KeyError: 'img_file'

In [121]:
0.125/50 * 100

0.25

In [9]:
H.root.forward(x_test) * np.array([1, 2, 3, 4, 5, 6, 7, 8])[:, None]

array([[0.46764591, 0.49666923],
       [0.94328308, 1.02254701],
       [1.21293542, 1.18626925],
       [2.11199021, 1.65704405],
       [2.94863075, 2.16121361],
       [3.21368623, 2.37944949],
       [4.61097169, 3.84166753],
       [4.8654685 , 4.40672684]])

In [25]:
x_test = next(iter(H.train_loader))

tensor(0., dtype=torch.float64)

In [88]:
H.dataset.flatten_labels["Pycnonotus"].shape

(445,)